# ELECCION DE HIPERPARAMETROS.
Analizamos 2 opciones, para quedarmos con el que nos entregue los mejores resultados de presicion:


##00-BIBLIOTECAS.

In [1]:
import requests
from io import StringIO
import seaborn as sns
import matplotlib.pyplot as plt
import plotly.express as px
import plotly.graph_objects as go
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler, OneHotEncoder
import pandas as pd
import numpy as np
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay
from sklearn.model_selection import cross_val_score as cv
from sklearn.model_selection import StratifiedKFold
from mlxtend.feature_selection import SequentialFeatureSelector as SFS
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import  precision_score

In [2]:
# Enlace al archivo CSV en GitHub
github_link = "https://raw.githubusercontent.com/Martinerramuspe/02-DATA_SCIENCIE/main/01-TELECOM_CHURN_PROJECT/02-PREPROCESAMIENTO/Prepro01.csv"
# Descargar el archivo CSV desde GitHub
response = requests.get(github_link)
csv_data = StringIO(response.text)
df = pd.read_csv(csv_data, encoding='utf-8')

In [3]:
# Definimos las caracteristicas segun lo ya definido en la etapa de eleccion de modelo.
X= df[[ 'International plan', 'Voice mail plan', 'Number vmail messages',
       'Total day minutes', 'Total day charge', 'Total eve minutes',
        'Total eve charge', 'Total night minutes', 'Total night charge',
        'Total intl minutes', 'Total intl calls']]

y=df.Churn

In [4]:
# Split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

##01-RANDOM FOREST CON HIPERPARAMETROS PREESTABLECIDOS.
Medimos la presicion construyendo un modelo (random forest) con los hiperparametros prestablecidos.

In [5]:
# Instanciamos.
forest_model = RandomForestClassifier()

In [6]:
# Fit.
forest_model.fit(X_train, y_train)

RandomForestClassifier()

In [7]:
# Observamos los hiperparametros predefinidos por el modelo.
forest_model.get_params()

{'bootstrap': True,
 'ccp_alpha': 0.0,
 'class_weight': None,
 'criterion': 'gini',
 'max_depth': None,
 'max_features': 'sqrt',
 'max_leaf_nodes': None,
 'max_samples': None,
 'min_impurity_decrease': 0.0,
 'min_samples_leaf': 1,
 'min_samples_split': 2,
 'min_weight_fraction_leaf': 0.0,
 'n_estimators': 100,
 'n_jobs': None,
 'oob_score': False,
 'random_state': None,
 'verbose': 0,
 'warm_start': False}

In [8]:
# Presicion en conjunto de entrenamiento.
y_pred_train = forest_model.predict(X_train)
# Determinamos precisión en el conjunto de entrenamiento.
precision_train = precision_score(y_train, y_pred_train, average='binary')  # 'binary' contempla el balance.
precision_train

1.0

In [9]:
# Presicion en conjunto de validacion.
skf = StratifiedKFold(n_splits=5, random_state=42, shuffle=True)
forest_cv_scores = cv(forest_model, X_train, y_train, cv=skf)
forest_cv_scores

array([0.95543175, 0.94707521, 0.9637883 , 0.96100279, 0.95530726])

In [10]:
# # Presicion en conjunto de testeo.
y_pred_test = forest_model.predict(X_test)
# Determinamos precisión en el conjunto de test.
precision_test = precision_score(y_test, y_pred_test, average='binary') # 'binary' contempla el balance.
precision_test

0.9714285714285714

##02-GRIDSEARCH CON RANGO DE HIPERPARAMETROS.
Medimos presicion utilizando distintos  rangos en los hiperparametros

In [11]:
# Definimos rangos para los distintos  hiperparámetros
param_grid = {
    'n_estimators': [75, 100, 150],
    'max_depth': [None, 20, 30],
    'min_samples_split': [1, 2],
    'min_samples_leaf': [1, 2],
    'bootstrap': [True],
    'ccp_alpha': [0.0],
    'class_weight': [None],
    'criterion': ['gini'],
    'max_features': ['sqrt'],
    'max_leaf_nodes': [None],
    'max_samples': [None],
    'min_impurity_decrease': [0.0],
    'min_weight_fraction_leaf': [0.0],
    'n_jobs': [None],
    'oob_score': [False],
    'random_state': [None],
    'verbose': [0],
    'warm_start': [False]
}

In [13]:
# Instanciamos GridSearchCV
grid_search = GridSearchCV(estimator=forest_model, param_grid=param_grid, cv=5, scoring='f1')  # Utilizamos "f1" para contemplar el balance.

In [14]:
# Fit
grid_search.fit(X_train, y_train)

/usr/local/lib/python3.10/dist-packages/sklearn/model_selection/_validation.py:378: FitFailedWarning: 
90 fits failed out of a total of 180.
The score on these train-test partitions for these parameters will be set to nan.
If these failures are not expected, you can try to debug them by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
90 fits failed with the following error:
Traceback (most recent call last):
  File "/usr/local/lib/python3.10/dist-packages/sklearn/model_selection/_validation.py", line 686, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "/usr/local/lib/python3.10/dist-packages/sklearn/ensemble/_forest.py", line 340, in fit
    self._validate_params()
  File "/usr/local/lib/python3.10/dist-packages/sklearn/base.py", line 600, in _validate_params
    validate_parameter_constraints(
  File "/usr/local/lib/python3.10/dist-packages/sklearn/util

GridSearchCV(cv=5, estimator=RandomForestClassifier(),
             param_grid={'bootstrap': [True], 'ccp_alpha': [0.0],
                         'class_weight': [None], 'criterion': ['gini'],
                         'max_depth': [None, 20, 30], 'max_features': ['sqrt'],
                         'max_leaf_nodes': [None], 'max_samples': [None],
                         'min_impurity_decrease': [0.0],
                         'min_samples_leaf': [1, 2],
                         'min_samples_split': [1, 2],
                         'min_weight_fraction_leaf': [0.0],
                         'n_estimators': [75, 100, 150], 'n_jobs': [None],
                         'oob_score': [False], 'random_state': [None],
                         'verbose': [0], 'warm_start': [False]},
             scoring='f1')

In [15]:
# Observamos los mejores hiperparametros logrados en el GridSearch.
grid_search.best_params_

{'bootstrap': True,
 'ccp_alpha': 0.0,
 'class_weight': None,
 'criterion': 'gini',
 'max_depth': 30,
 'max_features': 'sqrt',
 'max_leaf_nodes': None,
 'max_samples': None,
 'min_impurity_decrease': 0.0,
 'min_samples_leaf': 2,
 'min_samples_split': 2,
 'min_weight_fraction_leaf': 0.0,
 'n_estimators': 150,
 'n_jobs': None,
 'oob_score': False,
 'random_state': None,
 'verbose': 0,
 'warm_start': False}

In [17]:
# Traemos tabla, con las distintas combinaciones y sus resultado en el cross.
resultado=pd.DataFrame(grid_search.cv_results_)
resultado.sort_values("rank_test_score") # Ordenamos de forma decreciente.

,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_bootstrap,param_ccp_alpha,param_class_weight,param_criterion,param_max_depth,param_max_features,...,param_warm_start,params,split0_test_score,split1_test_score,split2_test_score,split3_test_score,split4_test_score,mean_test_score,std_test_score,rank_test_score
35,0.571246,0.016671,0.017706,0.001227,True,0.0,None,gini,30,sqrt,...,False,"{'bootstrap': True, 'ccp_alpha': 0.0, 'class_w...",0.828571,0.800000,0.738462,0.811594,0.727273,0.781180,0.040635,1
3,0.288080,0.016737,0.012092,0.002125,True,0.0,None,gini,None,sqrt,...,False,"{'bootstrap': True, 'ccp_alpha': 0.0, 'class_w...",0.861111,0.776119,0.757576,0.764706,0.746269,0.781156,0.041139,2
5,0.689949,0.139308,0.026814,0.005490,True,0.0,None,gini,None,sqrt,...,False,"{'bootstrap': True, 'ccp_alpha': 0.0, 'class_w...",0.861111,0.805970,0.738462,0.764706,0.727273,0.779504,0.048973,3
23,2.084532,0.818414,0.076378,0.025328,True,0.0,None,gini,20,sqrt,...,False,"{'bootstrap': True, 'ccp_alpha': 0.0, 'class_w...",0.861111,0.776119,0.738462,0.794118,0.727273,0.779416,0.047534,4
22,1.075104,0.578706,0.049410,0.034488,True,0.0,None,gini,20,sqrt,...,False,"{'bootstrap': True, 'ccp_alpha': 0.0, 'class_w...",0.861111,0.764706,0.746269,0.794118,0.727273,0.778695,0.046737,5
21,0.308726,0.002805,0.012008,0.000677,True,0.0,None,gini,20,sqrt,...,False,"{'bootstrap': True, 'ccp_alpha': 0.0, 'class_w...",0.861111,0.764706,0.764706,0.771429,0.727273,0.777845,0.044446,6
17,0.728033,0.134489,0.024411,0.004526,True,0.0,None,gini,20,sqrt,...,False,"{'bootstrap': True, 'ccp_alpha': 0.0, 'class_w...",0.845070,0.776119,0.746269,0.794118,0.727273,0.777770,0.040845,7
10,0.385771,0.018251,0.013245,0.000880,True,0.0,None,gini,None,sqrt,...,False,"{'bootstrap': True, 'ccp_alpha': 0.0, 'class_w...",0.845070,0.782609,0.738462,0.776119,0.746269,0.777706,0.037664,8
16,0.393209,0.006480,0.013702,0.000564,True,0.0,None,gini,20,sqrt,...,False,"{'bootstrap': True, 'ccp_alpha': 0.0, 'class_w...",0.861111,0.776119,0.757576,0.764706,0.727273,0.777357,0.044891,9
29,0.733782,0.126468,0.026361,0.005354,True,0.0,None,gini,30,sqrt,...,False,"{'bootstrap': True, 'ccp_alpha': 0.0, 'class_w...",0.861111,0.787879,0.727273,0.782609,0.727273,0.777229,0.049335,10
